# RecursiveJsonSplitter
- JSON 데이터를 깊이 우선 탐색하여 더 작은 JSON chunk를 생성한다.
- 되도록 중첩된 JSON 객체를 유지하려고 하지만, chunk 크기를 `min_chunk_size`와 `max_chunk_size` 사이를 유지하기 위해 중첩된 JSON 객체를 분할할 수 있다.
- JSON의 문자열 값이 매우 긴 경우엔 분할되지 않는다.

In [35]:
import requests

# JSON 데이터를 로드합니다.
json_data = requests.get("https://api.smith.langchain.com/openapi.json").json()
json_data

{'openapi': '3.1.0',
 'info': {'title': 'LangSmith', 'version': '0.1.0'},
 'paths': {'/api/v1/sessions/{session_id}': {'get': {'tags': ['tracer-sessions'],
    'summary': 'Read Tracer Session',
    'description': 'Get a specific session.',
    'operationId': 'read_tracer_session_api_v1_sessions__session_id__get',
    'security': [{'API Key': []}, {'Tenant ID': []}, {'Bearer Auth': []}],
    'parameters': [{'name': 'session_id',
      'in': 'path',
      'required': True,
      'schema': {'type': 'string', 'format': 'uuid', 'title': 'Session Id'}},
     {'name': 'include_stats',
      'in': 'query',
      'required': False,
      'schema': {'type': 'boolean',
       'default': False,
       'title': 'Include Stats'}},
     {'name': 'accept',
      'in': 'header',
      'required': False,
      'schema': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
       'title': 'Accept'}}],
    'responses': {'200': {'description': 'Successful Response',
      'content': {'application/json': {'sch

In [36]:
from langchain_text_splitters import RecursiveJsonSplitter

# JSON 데이터를 최대 300 크기의 청크로 분할하는 RecursiveJsonSplitter 객체를 생성합니다.
splitter = RecursiveJsonSplitter(max_chunk_size=300)

In [37]:
# JSON 데이터를 기반으로 문서를 생성합니다.
docs = splitter.create_documents(texts=[json_data])

# JSON 데이터를 기반으로 문자열 청크를 생성합니다.
texts = splitter.split_text(json_data=json_data)

# 첫 번째 문자열을 출력합니다.
print(docs[0].page_content)

print("===" * 20)
# 분할된 문자열 청크를 출력합니다.
print(texts[0])

{"openapi": "3.1.0", "info": {"title": "LangSmith", "version": "0.1.0"}, "paths": {"/api/v1/sessions/{session_id}": {"get": {"tags": ["tracer-sessions"], "summary": "Read Tracer Session", "description": "Get a specific session."}}}}
{"openapi": "3.1.0", "info": {"title": "LangSmith", "version": "0.1.0"}, "paths": {"/api/v1/sessions/{session_id}": {"get": {"tags": ["tracer-sessions"], "summary": "Read Tracer Session", "description": "Get a specific session."}}}}


- 3번째 chunk의 size가 300이 넘는걸 확인할 수 있는데, 이것은 **splitter가 리스트 객체를 분할하지 않기 때문이다**.
- `convert_lists=True`로 설정하여 리스트를 `index:item` 형태의 key-value 쌍으로 변환할 수 있다

In [38]:
print(len(texts[2]))
print(texts[2])

469
{"paths": {"/api/v1/sessions/{session_id}": {"get": {"parameters": [{"name": "session_id", "in": "path", "required": true, "schema": {"type": "string", "format": "uuid", "title": "Session Id"}}, {"name": "include_stats", "in": "query", "required": false, "schema": {"type": "boolean", "default": false, "title": "Include Stats"}}, {"name": "accept", "in": "header", "required": false, "schema": {"anyOf": [{"type": "string"}, {"type": "null"}], "title": "Accept"}}]}}}}


In [39]:
import json

data = json.loads(texts[2])
data["paths"]

{'/api/v1/sessions/{session_id}': {'get': {'parameters': [{'name': 'session_id',
     'in': 'path',
     'required': True,
     'schema': {'type': 'string', 'format': 'uuid', 'title': 'Session Id'}},
    {'name': 'include_stats',
     'in': 'query',
     'required': False,
     'schema': {'type': 'boolean',
      'default': False,
      'title': 'Include Stats'}},
    {'name': 'accept',
     'in': 'header',
     'required': False,
     'schema': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
      'title': 'Accept'}}]}}}

In [44]:
texts = splitter.split_text(json_data=json_data, convert_lists=True)
docs = splitter.create_documents(texts=[json_data], convert_lists=True)

In [45]:
print(len(texts[2]))
texts[2]

203


'{"paths": {"/api/v1/sessions/{session_id}": {"get": {"parameters": {"0": {"name": "session_id", "in": "path", "required": true, "schema": {"type": "string", "format": "uuid", "title": "Session Id"}}}}}}}'

In [47]:
len(docs[2].page_content)
docs[2]

Document(page_content='{"paths": {"/api/v1/sessions/{session_id}": {"get": {"parameters": {"0": {"name": "session_id", "in": "path", "required": true, "schema": {"type": "string", "format": "uuid", "title": "Session Id"}}}}}}}')